<a href="https://colab.research.google.com/github/nihalmascar/OCRusingDeepLearning/blob/main/OCRwithinBoudningBox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q ultralytics
!pip install -q EasyOCR
!pip3 install -q torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4

In [ ]:
!unzip -q /content/drive/MyDrive/my_model.zip -d /content/model

In [ ]:
model_path = "/content/model/my_model.pt"
test_image_path = "/content/drive/MyDrive/Vertical.jpg"

In [ ]:
from ultralytics import YOLO
model = YOLO(model_path)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
results = model.predict(source =  test_image_path, save=True, save_txt=True, show_labels=False, show_conf=False)


image 1/1 /content/drive/MyDrive/Vertical.jpg: 448x640 1 Date of birth, 1 Expiry Date, 1 ID Number, 1 Issue Date, 1 Name, 1 Nationality, 603.3ms
Speed: 15.8ms preprocess, 603.3ms inference, 35.8ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels


In [ ]:
def boundingbox(img_width,img_height,x_center,y_center,width,height):
  #Calculate the coordinates of the bounding box
  x_center_pixel = x_center * img_width
  y_center_pixel = y_center * img_height
  half_width = width * img_width / 2
  half_height = height * img_height / 2

  #Calculate the coordinates of the bounding box
  x_min = int(x_center_pixel - half_width)
  y_min = int(y_center_pixel - half_height)
  x_max = int(x_center_pixel + half_width)
  y_max = int(y_center_pixel + half_height)

  return x_min, y_min, x_max, y_max

In [ ]:
import os
folder_path = '/content/output_images'
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
else:
    print(f"Folder '{folder_path}' already exists.")

In [ ]:
import cv2
predicted_image = '/content/runs/detect/predict/Vertical.jpg'
image = cv2.imread(predicted_image)

text_file_path = '/content/runs/detect/predict/labels/Vertical.txt'

with open(text_file_path, 'r') as file:
    lines = file.readlines()

for i , line in enumerate(lines):
  parts = line.split()
  if len(parts) == 5:
    class_id, x_center, y_center, width, height = map(float, parts)

    img_width = image.shape[0]
    img_height = image.shape[1]
    x_min, y_min, x_max, y_max = boundingbox(img_width,img_height,x_center,y_center,width,height)

    image = cv2.resize(image, (img_width,img_height))
    cropped_region = image[y_min:y_max, x_min:x_max]

    cv2.imwrite(f'/content/output_images/cropped_image_{i}.jpg', cropped_region)


Performing OCR on the images

In [ ]:
import easyocr
import os
output = '/content/output_images'
reader = easyocr.Reader(['en'])
result = []
for image_path in os.listdir(output):
    input_path = os.path.join(output, image_path)
    result += reader.readtext(input_path)


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
for (bbox,text,prob) in result:
    print(f'{text}')

India
16/04/2003
784-2003-2021616-91
Nihal Nishanth Walter Mascarenhas
24/03/2023
23/03/2025
